In [2]:
import torch
import datasets
import numpy as np
from transformers import AutoModelForSequenceClassification, AutoTokenizer

In [3]:
local_model_path = 'models/facebook-bart-large-mnli/'

In [4]:
model = AutoModelForSequenceClassification.from_pretrained(local_model_path, local_files_only=True, num_labels=2, ignore_mismatched_sizes=True)

In [5]:
raw_datasets = datasets.load_dataset('csv', data_files={'test':'data/pnli_test_unlabeled.csv'}, column_names=['sentence1', 'sentence2', 'labels'])

Using custom data configuration default-7d31c80073d910f0
Reusing dataset csv (C:\Users\yashv\.cache\huggingface\datasets\csv\default-7d31c80073d910f0\0.0.0\433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519)


  0%|          | 0/1 [00:00<?, ?it/s]

In [6]:
tokenizer = AutoTokenizer.from_pretrained(local_model_path, local_files_only=True)

In [22]:
result = []
for i in range(0, len(raw_datasets['test']["sentence1"]), 100):
    sentence1 = raw_datasets['test']["sentence1"][i:i+100]
    sentence2 = raw_datasets['test']['sentence2'][i:i+100]
    tokenized_datasets = tokenizer(sentence1, sentence2, padding=True, 
                               truncation=True, return_tensors='pt')
    pred = model(**tokenized_datasets)
    pred = np.argmax(pred.logits.detach().numpy(), axis=1)
    result.extend(pred)

In [23]:
len(result)

4850

In [25]:
with open('upload_predictions.txt', 'w', encoding = 'utf-8') as fp:
    for x in result:
        fp.write(str(x) + '\n')

In [27]:
results = [int(x) for x in result]

In [28]:
import gc

gc.collect()

torch.cuda.empty_cache()

In [ ]:
import os
os._exit(00)